# Setup
Installing dependencies

In [ ]:
pip install -q PDFPlumber pandas openpyxl PyMuPDF

# Table 1

## Extraction

In [29]:
import pdfplumber
import pandas as pd
import re

with pdfplumber.open('/content/Thailand cosmetics guidelines.pdf') as pdf:
  table1 = pdf.pages[1:5]
  table2 = pdf.pages[5:8]

  table1_df = pd.DataFrame({"Type": [], "Substance": [], "Level": []})

  for page_seq, page in enumerate(table1):
    table = page.extract_table()
    print(table)

[['Type', 'Specially-controlled substances', 'Maximum Lvel/ allowed'], ['1. Products for use on\nscalp hair', '- oxalic acid including esters and\nalkaline salts of oxalic acid\n- resorcinol\n- hydrogen peroxide, including other\nperoxides except sodium peroxide', '- not more than 5.0%\n- not more than 0.5%\n- not more than 12.0%'], ['2. Hair Perming\nProduct s', '- thioglycolic acid and its salts\n- thioglycolic acid esters\n- thiolactic acid', '- not more than 11.0% at pH\n7-9.5 calculated in form of\nthioglycolic acid\n- not more than 11.0% at pH\n6-9.5 calculated in form of\nthioglycolic acid\n- not more than 8.5% at pH\nnot more than 9.5'], ['3. Hair Straightening', '- thioglycolic acid and its salts', '- not more than 11.0% at pH']]
[['Products', '- thioglycolic acid esters\n- potassium hydroxide or sodium\nhydroxide', '7-9.5 calculated in form of\nthioglycolic acid\n- not more than 11.0% at pH\n6-9.5 calculated in form of\nthioglycolic acid\n- not more than 2.0%\ncalculated in f

Here, we're using PDFPlumber to extract the table data from each page. Some cells overflow onto the next page. We can identify that a cell is contiguous, as all types begin with a number, e.g. "3. Hair Straightening Products."

If the type (value in index 0 of the row) does not begin with a number, we can assume it's part of the previous cell and we simply append it.

In [3]:
import pdfplumber
import pandas as pd
import re

with pdfplumber.open('/content/Thailand cosmetics guidelines.pdf') as pdf:
  table1 = pdf.pages[1:5]

  table1_data = []

  for page_seq, page in enumerate(table1):
    table = page.extract_table()
    for row_seq, row in enumerate(table):
      pattern = r"^\d+\."
      begins_with_number = re.match(pattern, row[0])

      if page_seq == 0 and row_seq == 0:
        # this is the header row, ignore
        continue

      elif len(table1_data) > 1 and not begins_with_number:
        prev_row = table1_data[-1]
        new_row = [prev_row[i] + "\n" + row[i] for i in range(3)]
        table1_data[-1] = new_row

      else:
        table1_data.append(row)

  print(table1_data)

[['1. Products for use on\nscalp hair', '- oxalic acid including esters and\nalkaline salts of oxalic acid\n- resorcinol\n- hydrogen peroxide, including other\nperoxides except sodium peroxide', '- not more than 5.0%\n- not more than 0.5%\n- not more than 12.0%'], ['2. Hair Perming\nProduct s', '- thioglycolic acid and its salts\n- thioglycolic acid esters\n- thiolactic acid', '- not more than 11.0% at pH\n7-9.5 calculated in form of\nthioglycolic acid\n- not more than 11.0% at pH\n6-9.5 calculated in form of\nthioglycolic acid\n- not more than 8.5% at pH\nnot more than 9.5'], ['3. Hair Straightening\nProducts', '- thioglycolic acid and its salts\n- thioglycolic acid esters\n- potassium hydroxide or sodium\nhydroxide', '- not more than 11.0% at pH\n7-9.5 calculated in form of\nthioglycolic acid\n- not more than 11.0% at pH\n6-9.5 calculated in form of\nthioglycolic acid\n- not more than 2.0%\ncalculated in form of sodium\nhydroxide'], ['4. Hair dyeing Products', '- diaminophenols\n- hy

## Transforming the data

1. Creating an SQLite database and defining the schema
2. Separating bullet points, creating separate records for each substance and the way it is regulated in a particular product type.
3. Committing changes

In [27]:
  import sqlite3

  conn_table1 = sqlite3.connect('t1.db')
  cursor = conn_table1.cursor()

  cursor.execute('''
    CREATE TABLE IF NOT EXISTS Types (
      ID INTEGER PRIMARY KEY,
      TypeName TEXT UNIQUE
    )
  ''')

  cursor.execute('''
    CREATE TABLE IF NOT EXISTS Substances (
      ID INTEGER PRIMARY KEY,
      SubstanceName TEXT UNIQUE
    )
  ''')

  cursor.execute('''
    CREATE TABLE IF NOT EXISTS Regulations (
      ID INTEGER PRIMARY KEY,
      TypeID INTEGER,
      SubstanceID INTEGER,
      MaxLevelRegulatoryWording TEXT,
      FOREIGN KEY (TypeID) REFERENCES Types(ID),
      FOREIGN KEY (SubstanceID) REFERENCES Substances(ID)
    )
  ''')

  for row in table1_data:
    type_number_regex = r'^\d+\.\s*'
    type_name_clear_newline = re.sub('\n', ' ', row[0])
    # Removing newlines (\n), as well as the number at the start
    type_name = re.sub(type_number_regex, '', type_name_clear_newline)


    # Inserting types into db
    cursor.execute('''
      INSERT OR IGNORE INTO Types (TypeName)
      VALUES (?)
    ''', (type_name,))

    type_id = cursor.lastrowid

    # Cleaning up the names of substances, ignoring
    # first element from the split, as Python outputs an array as if
    # there is an empty element before the "-" delimiter
    listed_substances = row[1].split('\n- ')
    listed_substances = [
        # Removing trailing spaces and replace newlines with spaces
        substance.strip().replace('\n', ' ')
        for substance
        in listed_substances
        ]

    listed_max_levels = row[2].split('\n- ')
    listed_max_levels = [level.strip().replace('\n', ' ') for level in listed_max_levels]

    for i, substance in enumerate(listed_substances):
      cursor.execute('''
        INSERT OR IGNORE INTO Substances (SubstanceName)
        VALUES (?)
      ''', (substance,))
      substance_id = cursor.lastrowid

      cursor.execute('''
        INSERT OR IGNORE INTO Regulations (TypeID, SubstanceID, MaxLevelRegulatoryWording)
        VALUES (?, ?, ?)
      ''', (type_id, substance_id, listed_max_levels[i]))


  conn.commit()
  conn.close()

## Exporting

In [29]:
  conn_table1 = sqlite3.connect('t1.db')
  cursor = conn_table1.cursor()

  # Types DataFrame
  cursor.execute('''
    SELECT * FROM Types
  ''')
  types_df = pd.DataFrame(cursor.fetchall(), columns=["ID", "TypeName"])

  # Substances DataFrame
  cursor.execute('''
    SELECT * FROM Substances
  ''')
  substances_df = pd.DataFrame(cursor.fetchall(), columns=["ID", "SubstanceName"])

  # Regulations DataFrame
  cursor.execute('''
    SELECT * FROM Regulations
  ''')
  regulations_df = pd.DataFrame(cursor.fetchall(), columns=["ID", "TypeID", "SubstanceID", "MaxLevelRegulatoryWording"])

  conn_table1.close()

  writer = pd.ExcelWriter('t1.xlsx')
  types_df.to_excel(writer, sheet_name='Types', index=False)
  substances_df.to_excel(writer, sheet_name='Substances', index=False)
  regulations_df.to_excel(writer, sheet_name='Regulations', index=False)
  writer.close()

# Table 2

In [332]:
import pymupdf
import sqlite3

conn_table2 = sqlite3.connect('t2.db')
cursor = conn_table2.cursor()

cursor.execute('''
  CREATE TABLE IF NOT EXISTS Types (
    ID INTEGER PRIMARY KEY,
    TypeName TEXT UNIQUE
  )
''')

cursor.execute('''
  CREATE TABLE IF NOT EXISTS Substances (
    ID INTEGER PRIMARY KEY,
    SubstanceName TEXT UNIQUE,
    TypeID INTEGER,
    FOREIGN KEY (TypeID) REFERENCES Types(ID)
  )
''')

cursor.execute('''
  CREATE TABLE IF NOT EXISTS ProductTypeRegulation (
    ID INTEGER PRIMARY KEY,
    TypeID INTEGER,
    ConditionRegulatoryWording TEXT,
    RelevantNotificationToMinisterOfPublicHealth TEXT,
    FOREIGN KEY (TypeID) REFERENCES Types(ID)
  )
''')

cursor.execute('''
  CREATE TABLE IF NOT EXISTS SubstanceRegulation (
    ID INTEGER PRIMARY KEY,
    SubstanceID INTEGER,
    MaxLevel TEXT,
    ConditionRegulatoryWording TEXT,
    RelevantNotificationToMinisterOfPublicHealth TEXT,
    FOREIGN KEY (SubstanceID) REFERENCES Substances(ID)
  )
''')

def get_blocks_in_cell(cell):
  blocks = page.get_text_blocks(clip=cell)
  return blocks

# Function which gets the lines of text (as they appear visually) in a cell
def get_lines(cell):
  blocks = get_blocks_in_cell(cell)
  lines_acc = []
  for block in blocks:
    lines_acc.extend(block[4:][0].split("\n"))
    # By filtering using the bool function, we are getting rid of falsy values
    # which arise due to excessive line-breaks and spaces
  return list(filter(bool, lines_acc))

def split_row_into_padded_subrows(row, include_notification=False, include_conditions=False):
  subrows = []
  substance_lines = get_lines(row.cells[1])
  max_volume_lines = get_lines(row.cells[2])
  condition_lines = get_lines(row.cells[3])
  notification_lines = get_lines(row.cells[4])

  current_subrow = 0

  passedon_substance = ""
  passedon_max_volume = ""

  for i in range(max(len(substance_lines), len(max_volume_lines), len(condition_lines))):
    # Getting all the values on the current line
    try:
      line_substance = substance_lines[i]
    except:
      line_substance = " "

    try:
      line_max_volume = max_volume_lines[i]
    except:
      line_max_volume = ""

    try:
      line_condition = condition_lines[i]
    except:
      line_condition = ""
      
    try:
      line_notification = notification_lines[i]
    except:
      line_notification = ""

    new_substance = False
    new_max_volume = False

    # If the current line begins with digits, followed by a dot, it is a
    # new substance
    pattern = r"\d+\.\s*"
    if re.match(pattern, line_substance):
      line_substance = re.sub(pattern, "", line_substance)
      passedon_substance = line_substance
      new_substance = True

    # If the max volume is not an empty line with a space, it is preserved
    # for the next empty line
    try:
      if line_max_volume.strip():
        passedon_max_volume = line_max_volume
        new_max_volume = True
    except:
      max_volume_lines.append("")

    if new_substance:
      passedon_max_volume = ""

    new_subrow = new_substance or (new_max_volume and line_substance == " ")

    if new_subrow:
      current_subrow = current_subrow + 1
      subrows.append((
        line_substance if line_substance != " " else passedon_substance,
        line_max_volume if line_max_volume != " " else passedon_max_volume,
        line_condition if include_conditions else "",
        line_notification if include_notification else ""
        ))
    else:
      subrows[current_subrow - 1] = (
        subrows[-1][0] + "\n" + line_substance,
        subrows[-1][1] + "\n" + line_max_volume,
        subrows[-1][2] + "\n" + line_condition if include_conditions else "",
        subrows[-1][3] + "\n" + line_notification if include_notification else ""
        )
      passedon_substance = passedon_substance + " " + line_substance
  return subrows

doc = pymupdf.open('/content/Thailand cosmetics guidelines.pdf')
pages = doc[5:9]

for page_seq, page in enumerate(pages):
  tabs = page.find_tables()
  data = tabs[0].extract()

  for row_seq, row in enumerate(tabs[0].rows):
    if row_seq == 0 and page_seq == 0:
      # this is the header row, ignore
      continue

    # Extract type
    type_number_regex = r'^\d+\.\s*'
    type_name = re.sub(type_number_regex, '', data[row_seq][0])
    type_name = re.sub('\n', ' ', type_name)

    cursor.execute('''
      INSERT OR IGNORE INTO Types (TypeName)
      VALUES (?)
    ''', (type_name.strip().replace("\n"," "),))
    type_id = cursor.lastrowid

    # Extract substances listed
    listed_substances_acc = []

    if listed_substances != "-":
      listed_substances_acc = re.split(r"\d+\.\s*", data[row_seq][1])
      listed_substances_acc = [substance.strip().replace("\n", " ") for substance in listed_substances_acc[1:]]

    # Extract notifications
    notifications = [notification.replace("\n", " ") for notification in re.findall(r"\(No. \d+\)\n?B\.E\.\s?\d{4}", data[row_seq][4])]

    # Extract conditions
    conditions = re.split(r"\n\s*\-\s*", data[row_seq][3])

    if len(listed_substances_acc) > 0:

      # Decide how to break down into subrows
      notification_per_substance = True
      conditions_present = False

      if len(notifications) == 1 and len(listed_substances_acc) > 1:
        # If there is more than 1 listed substance, but only a single notification
        # we can assume that the notification only pertains to the type of
        # cosmetic product, and we can leave it out of the subrow breakdown
        notification_per_substance = False

        cursor.execute('''
          INSERT OR IGNORE INTO ProductTypeRegulation (TypeID, RelevantNotificationToMinisterOfPublicHealth)
          VALUES (?, ?)
        ''', (type_id, notifications[0].strip().replace("\n"," ")))

      if len(conditions) > 0 and bool(conditions[0]):
        # If there are no conditions, we won't process that column when carrying
        # out the subrow breakdown
        conditions_present = True


      subrows = split_row_into_padded_subrows(row, notification_per_substance, conditions_present)

      for subrow in subrows:
        substance = subrow[0]
        max_volume = subrow[1]
        conditions = subrow[2]
        notification = subrow[3]

        cursor.execute('''
          INSERT OR IGNORE INTO Substances (SubstanceName, TypeID)
          VALUES (?, ?)
        ''', (substance.strip().replace("\n"," "), type_id))
        substance_id = cursor.lastrowid

        cursor.execute('''
          INSERT INTO SubstanceRegulation (SubstanceID, MaxLevel, ConditionRegulatoryWording, RelevantNotificationToMinisterOfPublicHealth)
          VALUES (?, ?, ?, ?)
        ''', (substance_id, max_volume.strip().replace("\n"," "), conditions.strip().replace("\n"," "), notification.strip().replace("\n"," ")))

    else:
      cursor.execute('''
        INSERT OR IGNORE INTO ProductTypeRegulation (TypeID, RelevantNotificationToMinisterOfPublicHealth)
        VALUES (?, ?)
      ''', (type_id, notifications[0].strip().replace("\n"," ")))

      conditions_acc = []
      remainder_are_contents = False
      for condition in conditions:
        condition = condition.strip().replace("- ", "").replace("\n", " ")
        if condition.endswith("contains"):
          remainder_are_contents = True
        if remainder_are_contents:
          conditions_acc[-1] = conditions_acc[-1] + "\n" + condition + "; "
        else:
          conditions_acc.append(condition)
      for condition in conditions_acc:
        cursor.execute('''
          INSERT OR IGNORE INTO ProductTypeRegulation (TypeID, ConditionRegulatoryWording)
          VALUES (?, ?)
        ''', (type_id, condition.strip().replace("\n"," ")))

conn_table2.commit()
conn_table2.close()

In [333]:
conn_table2 = sqlite3.connect('t2.db')
cursor = conn_table2.cursor()

cursor.execute('''
  SELECT * FROM Types
''')
types_df = pd.DataFrame(cursor.fetchall(), columns=["ID", "TypeName"])

cursor.execute('''
  SELECT * FROM Substances
''')
substances_df = pd.DataFrame(cursor.fetchall(), columns=["ID", "SubstanceName", "TypeID"])

cursor.execute('''
  SELECT * FROM ProductTypeRegulation
''')
product_type_regulations_df = pd.DataFrame(cursor.fetchall(), columns=["ID", "TypeID", "ConditionRegulatoryWording", "RelevantNotificationToMinisterOfPublicHealth"])

cursor.execute('''
  SELECT * FROM SubstanceRegulation
''')
substance_regulations_df = pd.DataFrame(cursor.fetchall(), columns=["ID", "SubstanceID", "MaxLevel", "ConditionRegulatoryWording", "RelevantNotificationToMinisterOfPublicHealth"])

conn_table2.close()

writer = pd.ExcelWriter('t2.xlsx')
types_df.to_excel(writer, sheet_name='Types', index=False)
substances_df.to_excel(writer, sheet_name='Substances', index=False)
product_type_regulations_df.to_excel(writer, sheet_name='ProductTypeRegulations', index=False)
substance_regulations_df.to_excel(writer, sheet_name='SubstanceRegulations', index=False)
writer.close()